# Weighted Average Ensemble

1. Install the packages

In [ ]:
%pip install -q tensorflow opencv-python numpy pandas  matplotlib  visualkeras keras pathlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 992.0/992.0 kB 55.7 MB/s eta 0:00:00


2. Import packages

In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import visualkeras
import pandas as pd
import numpy as np
import pathlib
from tensorflow import keras
import os
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


3.  Import data + 4. Defining data path and image parameters

In [ ]:
#data_dir = pathlib.Path(' /content/drive/My Drive/lab/project/HS/Dataset/Kvasir')
data_dir = pathlib.Path('/content/drive/My Drive/lab/project/HS/Trial/Kvasir')
img_height=224
img_width=224

In [ ]:
print(data_dir)

/content/drive/My Drive/lab/project/HS/Trial/Kvasir


5. Training set

In [ ]:
train_ds  =  tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=32)

Found 3936 files belonging to 5 classes.
Using 3149 files for training.


6. Validation set

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=32)

Found 3936 files belonging to 5 classes.
Using 3149 files for training.


7. Class names in the dataset

In [ ]:
class_names = train_ds.class_names
print(class_names) #5 classes

['normal-cecum', 'normal-pylorus', 'normal-z-line', 'polyps', 'ulcerative-colitis']


7. Normalizing the values, fetching the images and put it into the buffer for faster training process

In [ ]:
normalization_layer  =  tf.keras.layers.Rescaling(1./255)
import numpy as np
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

#Pixelvalues are now [0,1]

print(np.min(first_image), np.max(first_image))
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

0.0 1.0


## MODEL1 (DenseNet)



1. Developing a DenseNet model

In [ ]:
from keras.layers.pooling.global_average_pooling2d import GlobalAveragePooling2D
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MaxPooling2D,GlobalAveragePooling2D,Dense,Dropout,Conv2D
base_model = DenseNet201(   
    input_shape=(224,224,3),
    weights='imagenet',
    include_top=False)

#Freeze the first 10 layers
for layer in base_model.layers[:10]:
      layer.trainable =False
x=base_model.output
#x=Conv2D(128,3,3),activation='relu')(x)
#on the top of the model we add  the layers below:
x = GlobalAveragePooling2D()(x)
x=Dense(512,activation='relu')(x)
x=Dropout(0.3)(x)
predictions=Dense(5,activation='sigmoid')(x)  # 5 neurons - because we have 8 classes

model1=Model(inputs=base_model.inputs, outputs=predictions)

2. Compiling the first model

In [ ]:
model1.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy'])

3. Calling fit function to MODEL1

In [ ]:
history=model1.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1
)

/usr/local/lib/python3.9/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


99/99 [==============================] - 270s 994ms/step - loss: 0.3699 - accuracy: 0.8781 - val_loss: 74.4367 - val_accuracy: 0.3465


4. Saving the model

In [ ]:
#!mkdir -p saved_model
model1.save('/content/drive/My Drive/lab/project/HS/Trial/model1')

## MODEL2 (InceptionV3)

1. Developing the InceptionV3 model

In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, Conv2D
base_model = InceptionV3(
                         input_shape=(224,224,3),
                          weights='imagenet',
                          include_top=False)

#Freeze the first 10 layers
for layer in base_model.layers[:10]:
      layer.trainable =False
x=base_model.output
#x=Conv2D(128,3,3),activation='relu')(x)
#on the top of the model we add  the layers below:
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='relu')(x)
x=Dropout(0.3)(x)
predictions=Dense(5,activation='sigmoid')(x)  # 8 neurons - because we have 8 classes

model2=Model(inputs=base_model.inputs, outputs=predictions)

2. Compiling the second model

In [ ]:
model2.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

3. Calling fit function for MODEL2

In [ ]:
history=model2.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1
)

/usr/local/lib/python3.9/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


99/99 [==============================] - 83s 369ms/step - loss: 0.5668 - accuracy: 0.7876 - val_loss: 13.3641 - val_accuracy: 0.2540


4. Saving the model

In [ ]:
model2.save('/content/drive/My Drive/lab/project/HS/Trial/model2')

## MODEL3 (ResNet50)

1. Developing the ResNet50 model

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, Conv2D
base_model = InceptionV3(
                         input_shape=(224,224,3),
                          weights='imagenet',
                          include_top=False)

#Freeze the first 10 layers
for layer in base_model.layers[:10]:
      layer.trainable =False
x=base_model.output
#x=Conv2D(128,3,3),activation='relu')(x)
#on the top of the model we add  the layers below:
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='relu')(x)
x=Dropout(0.3)(x)
predictions=Dense(5,activation='sigmoid')(x)  # 8 neurons - because we have 8 classes

model3=Model(inputs=base_model.inputs, outputs=predictions)

2. Compiling the model

In [ ]:
model3.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

3. Calling fit function for MODEL3

In [ ]:
history=model3.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1
)

/usr/local/lib/python3.9/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


99/99 [==============================] - 172s 298ms/step - loss: 0.6655 - accuracy: 0.7491 - val_loss: 21.5537 - val_accuracy: 0.3338


4. Saving the model

In [ ]:
model3.save('/content/drive/My Drive/lab/project/HS/Trial/model3')

## Import relevant libraries

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average

m1= load_model('/content/drive/My Drive/lab/project/HS/Trial/model1')
m1=Model(inputs=m1.inputs, outputs=m1.outputs,
         name = 'model1')

m2= load_model('/content/drive/My Drive/lab/project/HS/Trial/model2')
m2=Model(inputs=m2.inputs, outputs=m2.outputs,
         name = 'model2')

m3= load_model('/content/drive/My Drive/lab/project/HS/Trial/model3')
m3=Model(inputs=m3.inputs, outputs=m3.outputs,
         name = 'model3')

models = [m1, m2, m3]

##Ensemble model

1. Creating Ensemble Model

In [ ]:
models = [m1, m2, m3]
model_input = tf.keras.Input(shape=(224,224,3))    #Input of Ensemble
model_outputs = [model(model_input) for model in models] #Output of Ensemble - Average of model's output
ensemble_output = tf.keras.layers.Average()(model_outputs)
ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)

2. Compiling Ensemble Model

In [ ]:
ensemble_model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

3. Fitting the Ensemble

In [ ]:
history=ensemble_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1
    )

99/99 [==============================] - 386s 2s/step - loss: 1.0415 - accuracy: 0.9050 - val_loss: 1.4054 - val_accuracy: 0.5014


4. Creating the Weighted Average Layer (Low or High weight can be given to models if the individual model's accuracy is too high or too low)

In [ ]:
class WeightedAverageLayer(tf.keras.layers.Layer):
          def __init__(self, w1, w2, w3, **kwargs):
            super(WeightedAverageLayer, self).__init__(**kwargs)
            self.w1 = w1  #weights of the models
            self.w2 = w2
            self.w3 = w3

          def call(self, inputs):
            return self.w1 * inputs[0] + self.w2 * inputs[1]+ self.w3*inputs[2]   #set different weights for different models to have better accuracy

In [ ]:
ensemble_output = WeightedAverageLayer(0.6, 0.3, 0.1)(model_outputs)  # different weights for different models
ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)

5. Compiling the Ensemble model

In [ ]:
from keras.losses import SparseCategoricalCrossentropy
ensemble_model.compile(
    optimizer='adam',
    loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics = ['accuracy']
)

6. Fitting the Ensemble model

In [ ]:
history=ensemble_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1
)

99/99 [==============================] - 356s 1s/step - loss: 0.3250 - accuracy: 0.8962 - val_loss: 7.4807 - val_accuracy: 0.3233
